In [ ]:
## Linear Regression ##
#1. Load Data
X=df_train.values[:,:] #Matrix of [midpoint age,percentage]
y=df_train_target.values[:,1] #Array of votes of 3 choices: 1. Republican, 2. Democrat, 3. Other
print(X,y)

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.chdir('C:/Users/louka/Desktop/Machine Learning course/Project/Machine-learning-project')
print(os.getcwd())

C:\Users\louka\Desktop\Machine Learning course\Project\Machine-learning-project


In [52]:
df_train = pd.read_csv('Data/2016 Census AGE, RACE, LITERACY.csv')
df_train_target= pd.read_csv('Data/2016 Presidential Election Results by State.csv')
# df_test = pd.read_csv('Wyoming Census data 2020.csv')
# df_test_target=pd.read_csv('Wyoming total votes 2020.csv')
# #df_test.drop(0,axis=0,inplace=True)
# print(df_train_target)
# df_train=df_train
# df_test=df_test
df_train_target=df_train_target.dropna() #weird issue encountered
print(df_train_target)
print(df_train)


             STATE  Percentage Democrat  Percentage Republican  \
0          Alabama                 0.34                   0.62   
1           Alaska                 0.37                   0.51   
2          Arizona                 0.45                   0.49   
3         Arkansas                 0.34                   0.61   
4       California                 0.62                   0.32   
5         Colorado                 0.48                   0.43   
6      Connecticut                 0.55                   0.41   
7         Delaware                 0.53                   0.42   
8    Dist. of Col.                 0.91                   0.04   
9          Florida                 0.48                   0.49   
10         Georgia                 0.46                   0.51   
11          Hawaii                 0.62                   0.30   
12           Idaho                 0.28                   0.59   
13        Illinois                 0.56                   0.39   
14        

In [54]:
#useful functions
def swap_rows(df, row1, row2):
    df.iloc[row1], df.iloc[row2] =  df.iloc[row2].copy(), df.iloc[row1].copy()
    return df

In [56]:
df_train.set_index("Geographic Area Name",inplace=True) #nice
df_train_target.set_index("STATE",inplace=True)

In [58]:
print(df_train.loc["Alabama"],'\n')
print(df_train_target.loc["Alabama"])

Mean AGE                                                                                    37.8205
Percent under 19 years                                                                      25.4000
Percent between 19-45 years                                                                 38.6000
Percent between 45-65                                                                       26.5000
Percent over 65                                                                             14.4000
Percent RACE One  RACE White                                                                68.2000
Percent RACE One  RACE Black or African American                                            26.8000
Percent RACE One  RACE American Indian and Alaska Native                                     0.5000
Percent RACE One  RACE Asian                                                                 1.3000
Percent HISPANIC OR LATINO AND  RACE Total population Hispanic or Latino (of any  RACE )     4.1000


In [60]:
#Perform Kfold in 2016 (train and train_target) data
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
#Set X and y
X=df_train
y=df_train_target
print(y.shape)
k= 7 #number of folds. 7 is a good candidate for a balanced R^2 mean.
kf = KFold(n_splits=k, shuffle=False) #make kfold item
R2=[]
for (train_index, test_index) in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #print(X_train,X_test)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(X_test.index)
    reg=LinearRegression()
    reg.fit(X_train,y_train)
    y_pred=reg.predict(X_test)
    print('R^2= %.2f' %r2_score(y_test,y_pred))
    R2.append(r2_score(y_test,y_pred))
    print(y_pred,'\n')
R2_mean=np.mean(R2)
print('The mean R² score is: %.2f' %R2_mean)

(51, 3)
Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware'],
      dtype='object', name='Geographic Area Name')
R^2= -0.20
[[0.44819963 0.51259482 0.02357997]
 [0.33220105 0.56818806 0.10664019]
 [0.42988412 0.47983757 0.04421646]
 [0.35634062 0.60006837 0.02840666]
 [0.49082521 0.43142764 0.02861034]
 [0.53008832 0.31770582 0.05561982]
 [0.53913169 0.3852342  0.04112609]
 [0.41298506 0.57907165 0.01881376]] 

Index(['District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa'],
      dtype='object', name='Geographic Area Name')
R^2= -6.82
[[ 0.61968675  0.36426729  0.01051394]
 [ 0.57774922  0.34149022  0.04258821]
 [ 0.42735627  0.53779644  0.02011502]
 [ 1.53170001 -1.46099437 -0.01935809]
 [ 0.23985364  0.69677532  0.04268545]
 [ 0.45645652  0.49447217  0.03512235]
 [ 0.36844307  0.56197868  0.03169243]
 [ 0.34600537  0.54736154  0.04224997]] 

Index(['Kansas', 'Kentucky', 'Louisiana

In [62]:
reg.coef_

array([[-1.49133970e-01, -9.32187006e-02, -2.21650207e-02,
         1.63442379e-02,  1.06430156e-02, -1.15570546e-02,
        -9.54805085e-03, -1.89327564e-02, -2.51261857e-02,
        -2.22488820e-04,  3.34801799e-02,  1.35266913e-02,
        -1.66106586e-03,  5.59879715e-03, -1.47240104e-02,
        -8.33546044e-03],
       [ 1.80071233e-01,  1.23122560e-01,  4.29732288e-02,
        -1.96152975e-03,  8.71299142e-03,  1.29130568e-02,
         1.19371772e-02,  1.90195294e-02,  3.11981022e-02,
         4.71775864e-04, -4.25490778e-02, -1.34736584e-02,
         1.09905443e-05, -3.65348457e-03,  1.85725560e-02,
         7.79456559e-03],
       [ 1.16873029e-02,  7.74133874e-04, -9.48946628e-04,
        -3.52200954e-03, -7.44411000e-03, -2.49178511e-04,
        -9.07395365e-04,  2.16285805e-03, -2.61577701e-03,
         1.46032482e-04,  9.07565986e-03,  4.85831033e-04,
        -1.62294825e-03, -1.92049984e-04,  3.59429631e-04,
         1.40259450e-03]])